In [1]:
options(repos = structure(c(CRAN = "http://cran.ism.ac.jp/")))

In [4]:
# Generate a sample dataset
set.seed(1014)
df <- data.frame(replicate(6, sample(c(1:10, -99), 6, rep = TRUE)))
names(df) <- letters[1:6]
df

a,b,c,d,e,f
1,6,1,5,-99,1
10,4,4,-99,9,3
7,9,5,4,1,4
2,9,3,8,6,8
1,10,5,9,8,6
6,2,1,3,8,5


In [6]:
replicate(6, sample(c(1:10, -99), 6, rep = TRUE))

8,8,-99,6,10,-99
-99,8,5,9,4,6
2,2,6,3,7,6
3,9,4,8,4,4
8,9,2,1,7,5
6,-99,6,4,3,5


In [7]:
fix_missing <- function(x) {
  x[x == -99] <- NA
  x
}
df[] <- lapply(df, fix_missing)

In [8]:
df

a,b,c,d,e,f
1,6,1,5,NA,1
10,4,4,NA,9,3
7,9,5,4,1,4
2,9,3,8,6,8
1,10,5,9,8,6
6,2,1,3,8,5


In [11]:
library(dplyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [12]:
df$a %>% length


[1] 6

In [15]:
tmp = df

In [18]:
data.frame(lapply(df , length))

a,b,c,d,e,f
6,6,6,6,6,6


In [17]:
tmp

a,b,c,d,e,f
6,6,6,6,6,6
6,6,6,6,6,6
6,6,6,6,6,6
6,6,6,6,6,6
6,6,6,6,6,6
6,6,6,6,6,6


In [19]:
missing_fixer <- function(na_value) {
  function(x) {
    x[x == na_value] <- NA
    x
  }
}
fix_missing_99 <- missing_fixer(-99)
fix_missing_999 <- missing_fixer(-999)

fix_missing_99(c(-99, -999))

[1]   NA -999